In [10]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import kwant

ModuleNotFoundError: No module named 'matplotlib'

In [3]:
conda install qutip

^C

Note: you may need to restart the kernel to use updated packages.


In [ ]:
conda install -c conda-forge kwant

In [13]:
conda install -c pytorch pytorch-cuda

^C

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
device=torch.device('cuda')

In [15]:
a = torch.arange(60.).reshape(3, 4, 5)
b = torch.arange(24.).reshape(4, 3, 2)

In [4]:
torch.tensordot(a, b, dims=([1, 0], [0, 1]))

tensor([[4400., 4730.],
        [4532., 4874.],
        [4664., 5018.],
        [4796., 5162.],
        [4928., 5306.]])

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
device

device(type='cpu')

In [4]:
torch.cuda.is_available()

True

In [11]:
n_particles = 16
r = torch.rand((2,n_particles)).to(device)

In [16]:
torch.cuda.memory_stats()

OrderedDict([('active.all.allocated', 424),
             ('active.all.current', 25),
             ('active.all.freed', 399),
             ('active.all.peak', 28),
             ('active.large_pool.allocated', 58),
             ('active.large_pool.current', 0),
             ('active.large_pool.freed', 58),
             ('active.large_pool.peak', 2),
             ('active.small_pool.allocated', 366),
             ('active.small_pool.current', 25),
             ('active.small_pool.freed', 341),
             ('active.small_pool.peak', 28),
             ('active_bytes.all.allocated', 5193980928),
             ('active_bytes.all.current', 590336),
             ('active_bytes.all.freed', 5193390592),
             ('active_bytes.all.peak', 2684945408),
             ('active_bytes.large_pool.allocated', 5179965440),
             ('active_bytes.large_pool.current', 0),
             ('active_bytes.large_pool.freed', 5179965440),
             ('active_bytes.large_pool.peak', 2684354560),
          

In [17]:
torch.cuda.max_split_size_mb 

AttributeError: module 'torch.cuda' has no attribute 'max_split_size_mb'

In [5]:
def Jordan_wigner_transform(j, lattice_length,device):
    """Jordan_wigner_transform(j, lattice_length). 
Defines the Jordan Wigner transformation for a 1D lattice."""
    operators = torch.tensor([[1,0],[0,-1]],device=device)
    for _ in range(j-1): operators = torch.kron(operators, torch.tensor([[1,0],[0,-1]],device=device))
    if j == 0: operators = torch.tensor([[0,0],[1,0]],device=device)
    else: operators = torch.kron(operators, torch.tensor([[0,0],[1,0]],device=device))
    for _ in range(lattice_length - j - 1): operators = torch.kron(operators, torch.tensor([[1,0],[0,1]],device=device))
    return operators

def Lorentzian(omega, Gamma, poles,Ed=-3/2,Sigma=3/2):
    """Lorentzian(omega, Gamma, poles,Ed=-3/2,Sigma=3/2). 
Defines the non-interacting DOS (rho0) and selects random sites based on the number of sites in the 1D lattice model and the calculated distribution."""
    p = torch.rand(poles)
    return -np.imag(1/(omega-Ed-Sigma+1j*Gamma))/np.pi, np.array([Gamma * np.tan(np.pi * (p[i] - 1 / 2))+Ed+Sigma for i in range(poles)])

In [16]:
poles=6
U=3
Ed=-3/2
Sigma=3/2
Gamma=0.3
torch.tensor([Gamma * torch.tan(np.pi * (pi - 1 / 2))+Ed+Sigma for pi in torch.rand(poles)],device=device)

tensor([-0.0653, -0.0449,  0.2221,  0.0385,  0.1308,  0.8144], device='cuda:0')

In [38]:
p

tensor([0.9090, 0.4626, 0.1245, 0.2387, 0.7980, 0.2960])

In [34]:
poles=6
c=[Jordan_wigner_transform(i, 2*poles,device) for i in range(2*poles)]

In [56]:
poles=8
c=[Jordan_wigner_transform(i, 2*poles) for i in range(2*poles)]

KeyboardInterrupt: 

In [53]:
c

[tensor(indices=tensor([[2048, 2049, 2050,  ..., 4093, 4094, 4095],
                        [   0,    1,    2,  ..., 2045, 2046, 2047]]),
        values=tensor([1, 1, 1,  ..., 1, 1, 1]),
        size=(4096, 4096), nnz=2048, layout=torch.sparse_coo),
 tensor(indices=tensor([[1024, 1025, 1026,  ..., 4093, 4094, 4095],
                        [   0,    1,    2,  ..., 3069, 3070, 3071]]),
        values=tensor([ 1,  1,  1,  ..., -1, -1, -1]),
        size=(4096, 4096), nnz=2048, layout=torch.sparse_coo),
 tensor(indices=tensor([[ 512,  513,  514,  ..., 4093, 4094, 4095],
                        [   0,    1,    2,  ..., 3581, 3582, 3583]]),
        values=tensor([1, 1, 1,  ..., 1, 1, 1]),
        size=(4096, 4096), nnz=2048, layout=torch.sparse_coo),
 tensor(indices=tensor([[ 256,  257,  258,  ..., 4093, 4094, 4095],
                        [   0,    1,    2,  ..., 3837, 3838, 3839]]),
        values=tensor([ 1,  1,  1,  ..., -1, -1, -1]),
        size=(4096, 4096), nnz=2048, layout=torch.s

In [31]:
crow_indices = [0, 1, 2]
col_indices = [0,1]
values = [1,-1]
csr=torch.sparse_csr_tensor(torch.tensor(crow_indices, dtype=torch.int64),torch.tensor(col_indices, dtype=torch.int64),torch.tensor(values), dtype=torch.double)

In [39]:
torch.tensor([[1,0],[0,-1]])[1]

tensor([ 0, -1])

In [36]:
sigmaz=torch.tensor([[1,0],[0,-1]])
torch.kron(sigmaz, sigmaz)

tensor([[ 1,  0,  0,  0],
        [ 0, -1,  0,  0],
        [ 0,  0, -1,  0],
        [ 0,  0,  0,  1]])

In [16]:
torch.sparse_csr_tensor(torch.tensor([0,2,2], dtype=torch.int64),torch.tensor([0,1], dtype=torch.int64),torch.tensor([1,-1]))

RuntimeError: (Batch element 0) last value of crow_indices should be equal to the length of col_indices.

In [3]:
a,b

(tensor([[[ 0.,  1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.,  9.],
          [10., 11., 12., 13., 14.],
          [15., 16., 17., 18., 19.]],
 
         [[20., 21., 22., 23., 24.],
          [25., 26., 27., 28., 29.],
          [30., 31., 32., 33., 34.],
          [35., 36., 37., 38., 39.]],
 
         [[40., 41., 42., 43., 44.],
          [45., 46., 47., 48., 49.],
          [50., 51., 52., 53., 54.],
          [55., 56., 57., 58., 59.]]]),
 tensor([[[ 0.,  1.],
          [ 2.,  3.],
          [ 4.,  5.]],
 
         [[ 6.,  7.],
          [ 8.,  9.],
          [10., 11.]],
 
         [[12., 13.],
          [14., 15.],
          [16., 17.]],
 
         [[18., 19.],
          [20., 21.],
          [22., 23.]]]))